In [7]:
import cv2
import mediapipe as mp
import pickle
import numpy as np

In [10]:
# Import the model into this notebook
classifer_dict = pickle.load(open('./classifier_model.p', 'rb'))
classifer = classifer_dict['model']

In [15]:
capture = cv2.VideoCapture(0)

# Define the objects used to landmark the hands in the images
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Define the hand detector
hands = mp_hands.Hands(static_image_mode = True, min_detection_confidence=0.3)

labels_dict = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7:'H',
              8: 'I', 9: 'J', 10: 'K', 11: 'L', 12:'M', 13: 'N', 14:'O', 15: 'P',
              16: 'Q', 17: 'R', 18:'S', 19:'T', 20:'U', 21:'V', 22: 'W', 23: 'X',
              24: 'Y', 25: 'Z'}

while True:
    data_aux = []

    x_ = []
    y_ = []

    ret, frame = capture.read()

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Use the hand detection object on the image that is read-in
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        # Loop through all the landmarks made for each image from the
        # hand detection object
        for hand_landmarks in results.multi_hand_landmarks:
            #hand_landmarks = results.multi_hand_landmarks[0]
            # Draw the landmarks on the images (you will see them 
            # printed out on the image)
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

            # Loop through all the coordinates for each landmark
            for i in range(len(hand_landmarks.landmark)):
                
                # Get the x-coordinate
                x = hand_landmarks.landmark[i].x
                # Get y-coordinate
                y = hand_landmarks.landmark[i].y

                # Append the x and y coordinates for the specific landmark
                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10

        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10

        prediction = classifer.predict([np.asarray(data_aux)])

        predicted_character = (prediction[0])

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                    cv2.LINE_AA)

    cv2.imshow('frame', frame)
    cv2.waitKey(25)

capture.release()
cv2.destroyAllWindows()

I0000 00:00:1764383353.316322 2602914 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1
W0000 00:00:1764383353.332021 2640292 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764383353.346041 2640289 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


KeyboardInterrupt: 

: 